In [143]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy



if (not os.path.isfile('data/spy-ohlc.csv')):
    start = datetime.datetime(1900, 2, 1)
    end = datetime.datetime.now()
    df_train = data.DataReader("SPY", 'yahoo', start, end)
    df_train.to_csv("data/spy-ohlc.csv")

df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
latestDate = datetime.datetime.now() - datetime.timedelta(days=1)
if (df_train.index[-1] < latestDate):
    start = df_train.index[-1] + datetime.timedelta(days=1)
    new_data = data.DataReader("SPY", 'yahoo', start, latestDate)
    df_train.append(new_data)

df_train.to_csv("data/spy-ohlc.csv")

In [144]:
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)

df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [145]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()

df_train = df_train.assign(pma10=pma10)
df_train = df_train.assign(pma20=pma20)
df_train = df_train.assign(pma50=pma50)
df_train = df_train.assign(pma200=pma200)

In [146]:
diffma10 = (df_train['Close']/df_train['pma10'] -1)*100
diffma20 = (df_train['Close']/df_train['pma20'] -1)*100
diffma50 = (df_train['Close']/df_train['pma50'] -1)*100
diffma200 = (df_train['Close']/df_train['pma200'] -1)*100

df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

In [147]:
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)


In [148]:
hh = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hl = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 

df_train = df_train.assign(hh=hh)
df_train = df_train.assign(hl=hl)

hh10 = hh.rolling(window=10, center=False).mean()
hl10 = hl.rolling(window=10, center=False).mean()

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

In [149]:
df_train.to_csv("data/spy-train.csv")

In [190]:
y_train = hl.shift(-1).values[201:]

In [191]:
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend', 
          'pma10','pma20','pma50','pma200',
          'hh','hl',
          
          #diffma10,diffma20,diffma50,diffma200,
          #pma10tan,pma20tan,pma50tan,pma200tan,
          #'hh-10','hl-10'
         ]
X_train = df_train.drop(remove, axis=1).values[201:, :]
X_test = X_train[-1,:]
X_train = X_train[:-1, :]
print(len(X_train))

5631


In [192]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM

In [193]:
X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.25)

#clf = OneClassSVM();
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=150)
#from sklearn.svm import SVC
#model = SVC(probability=True)

# fitting
model.fit(X_fit, y_fit.astype(bool));
y_eval_pred = model.predict_proba(X_eval);


In [194]:
print(y_eval_pred[0:20,:])
print(y_eval[0:20])
y_eval_comp = y_eval_pred[:,1] > 0.4
print(y_eval_comp[0:20])
numpy.count_nonzero(y_eval_comp)

[[ 0.52        0.48      ]
 [ 0.51333333  0.48666667]
 [ 0.44        0.56      ]
 [ 0.68        0.32      ]
 [ 0.31333333  0.68666667]
 [ 0.39333333  0.60666667]
 [ 0.43333333  0.56666667]
 [ 0.52666667  0.47333333]
 [ 0.58666667  0.41333333]
 [ 0.57333333  0.42666667]
 [ 0.58666667  0.41333333]
 [ 0.29333333  0.70666667]
 [ 0.5         0.5       ]
 [ 0.54        0.46      ]
 [ 0.44666667  0.55333333]
 [ 0.52        0.48      ]
 [ 0.27333333  0.72666667]
 [ 0.57333333  0.42666667]
 [ 0.58666667  0.41333333]
 [ 0.44666667  0.55333333]]
[False True True False True True False False True True False True True
 False False False False False False False]
[ True  True  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


1215

In [195]:
print('Overall AUC:', roc_auc_score(y_eval,y_eval_pred[:,1]))

ValueError: unknown format is not supported

In [196]:
from sklearn import metrics

print(metrics.recall_score(y_eval, y_eval_comp, average='binary'))
print(metrics.precision_score(y_eval, y_eval_comp, average='binary'))
print(metrics.f1_score(y_eval, y_eval_comp, average='binary'))

ValueError: Can't handle mix of unknown and binary

In [197]:
y_test_pred = model.predict_proba(X_test);
print(y_test_pred)

[[ 0.52666667  0.47333333]]


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [117]:
#print(hh10)